In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# visualisation libraries
from matplotlib import pyplot as plt
import seaborn as sns
from numpy.random import RandomState


#word cloud
%matplotlib inline
import wordcloud

from wordcloud import WordCloud, STOPWORDS
%matplotlib inline
sns.set()

# visualisation libraries
from matplotlib import pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)


# ML Models
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ML Pre processing
from surprise.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Hyperparameter tuning
from surprise.model_selection import GridSearchCV

# High performance hyperparameter tuning
#from tune_sklearn import TuneSearchCV
#import warnings
#warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from wordcloud import WordCloud, STOPWORDS
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
        
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
import time
from surprise import SVD
from surprise import accuracy
import re
import plotly.express as px
import scipy as sp
from wordcloud import WordCloud, STOPWORDS
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline  import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, Normalizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
import gc
gc.collect()


### Table of contents:
1. Introduction
2. Import libaries and datasets
3. Exploratory Data Analysis
Summary statistics
Visualizing the dataframes
Visualizing the null values for each dataframe
Visualizing common users
Exploring movie genres
Exploring the movies dataframe
Word cloud
Publishing years
Budget
4. Prepocessing
5. Modelling
Content-Based Filtering Recommendation
Collaborative-Based Filtering Reccomendation
6. Evaluation
7. Conclusion

## Loading data

In [ ]:
#Pandas libraries used in the notebook.
train = pd.read_csv('../input/edsa-movie-recommendation-2022/train.csv')
test_df = pd.read_csv('../input/edsa-movie-recommendation-2022/test.csv')
df_movies = pd.read_csv('../input/edsa-movie-recommendation-2022/movies.csv')
df_samp = pd.read_csv('../input/edsa-movie-recommendation-2022/sample_submission.csv')
df_imdb = pd.read_csv('../input/edsa-movie-recommendation-2022/imdb_data.csv')
genome_tags = pd.read_csv("../input/edsa-movie-recommendation-2022/genome_tags.csv")
genome_score = pd.read_csv("../input/edsa-movie-recommendation-2022/genome_scores.csv")
df_tags = pd.read_csv("../input/edsa-movie-recommendation-2022/tags.csv")
df_links = pd.read_csv("../input/edsa-movie-recommendation-2022/links.csv")

### Exploratory Data Analysis

##### Checking the shapes of our Datasets

In [ ]:
print('train data:',train.shape) 
print('test data:',test_df.shape)
print('tags data:',df_tags.shape)
print("Movies data:",df_movies.shape)
print('links data:',df_links.shape)
print('imdb data:',df_imdb.shape)
print('genome tags data:',genome_tags.shape)
print('genome scores data:',genome_score.shape)

### Let us Visualize our dataframe

##### First, we would be calculating the sizes of our data, which would we compare using a Piechart and a Barplot

In [ ]:
"""" Creating a DataFrame which would outline the sizes of our data
"""

D_F = ['train', 'test_df', 'df_tags', 'df_imdb','links_df', 'df_movies', 'genome_tags', 'genome_score']

sizes = [len(train), len(test_df), len(df_tags),
         len(df_imdb), len(df_links), len(df_movies),
         len(genome_tags), len(genome_score)]
total_size_df = pd.DataFrame(list(zip(D_F, sizes)),
                             columns=['dataframe', 'sizes'])
total_size_df

Now let us separate the sizes into categorie, with sizes above 100000 and those below 100000 into category **Other** 

In [ ]:
total_size_df = total_size_df[total_size_df['sizes'] > 100000]
total_size_df

In [ ]:
new_row = {'dataframe': 'other', 'sizes': 180530}
total_size_df = total_size_df.append(new_row, ignore_index=True)
total_size_df

The DataFrames that had a larger size than 100000 include the train_df; test_df; tags_df; genome_score and other. 

While the DataFrame's that had a size smaller than 100000 were combined into a DataFrame 'other', these DataFrames are imdb_df; links_df, movies_df; and genome_tags.

In [ ]:
# explodeTuple = (0.05, 0.04, 0.05, 0.04, 0.6)
# fig1, ax1 = plt.subplots(figsize=(14,7))
# ax1.pie(total_size_df['sizes'].values,
#         labels=total_size_df['dataframe'].values,
#         startangle=90, autopct='%1.1f%%',
#         explode=explodeTuple)
# ax1.axis('equal')
# plt.title('Distribution of overall Data Frames')
# plt.show()

In [ ]:
len_list = [['train', len(train)], ['df_tags', len(df_tags)],
            ['df_imdb', len(df_imdb)], ['df_links', len(df_links)],
            ['df_movies', len(df_movies)],
            ['genome_tags', len(genome_tags)],
            ['genome_score', len(genome_score)]]
len_df = pd.DataFrame(len_list,
                      columns=['Dataset', 'Size'])
fig = px.bar(len_df, x=len_df['Dataset'],
             y=len_df['Size'],
             color=len_df['Dataset'],
             title='Distribution of overall Data Frames')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

Observation:
In the bar graph, there is a clear uneven distribution of the data sizes in each dataset
The genome_score DataFrame has the largest size, followed by the train_df.     

The difference in distribution sizes is clear, where the other DataFrame's bars aren't visually evident because of the large difference between the dimension

##### Let us Visualize Null Values present

In [ ]:
# First we would obtain the total null values in each Data Frames columns

train_count = pd.DataFrame(train.isnull().sum())
test_count = pd.DataFrame(test_df.isnull().sum())
tags_count = pd.DataFrame(df_tags.isnull().sum())
movies_count = pd.DataFrame(df_movies.isnull().sum())
links_count = pd.DataFrame(df_links.isnull().sum())
imdb_count = pd.DataFrame(df_imdb.isnull().sum())
genomet_count = pd.DataFrame(genome_tags.isnull().sum())
genomes_count = pd.DataFrame(genome_score.isnull().sum())



Fromm the output above, we can see that only **df_tags** and **df_imdb** show null values.

Let us visualize thise for better clarity

In [ ]:
plt.figure(figsize=[7,5])
plt.bar(tags_count.index,
        tags_count.values.reshape(len(tags_count), ),
        color='red')
plt.xlabel('column_name')
plt.ylabel('count')
plt.title('Null value count in tags_df')
plt.show()

In [ ]:
plt.figure(figsize=[7,5])
plt.bar(imdb_count.index,
        imdb_count.values.reshape(len(imdb_count), ),
        color=['orange','blue','green','black','purple'])
plt.xlabel('column_name')
plt.ylabel('count')
plt.title('Null value count in imdb_df')
plt.show()

### Now we will visualize common users

We would try to calculate the number of times a user rated a movie by first creating a dataframe that matches a count by userId

In [ ]:
##Create DataFrame with count by userID
user_count = pd.DataFrame(train['userId'].value_counts()).reset_index()
user_count.rename(columns={'index':'userId','userId':'count'},
                  inplace=True)
user_count.head()

Next, we will group the users within a range, by doing so, we can differentiate between common users and new ones



In [ ]:
## Grouping users by count within range
first_group = user_count.loc[(user_count['count'] > 0) &  #between 1 - 50
            (user_count['count'] < 50),
            'userId'].value_counts().sum()
second_group = user_count.loc[(user_count['count'] >= 50) &  #between 50 - 500
            (user_count['count'] < 500),
            'userId'].value_counts().sum()
third_group = user_count.loc[(user_count['count'] >= 500) & #between 500 - 1000
            (user_count['count'] < 1000),
            'userId'].value_counts().sum()
fourth_group = user_count.loc[(user_count['count'] >= 1000) & #between 1000 - 1500
            (user_count['count'] < 1500),
            'userId'].value_counts().sum()
fifth_group = user_count.loc[(user_count['count'] >= 1500),#from 1500 above
            'userId'].value_counts().sum()

In [ ]:
# To give us insight in the spread, we used figures to determine the spread.
trial_error = np.array([['first_group', first_group,
                         'between 1 and 50'],
                        ['second_group', second_group,
                         'between 50 and 500'],
                        ['third_group', third_group,
                         'between 500 and 1000'],
                        ['fourth_group', fourth_group,
                         'between 1000 and 1500'],
                        ['fifth_group', fifth_group,
                         'greater than 1500']])


trial_error_df = pd.DataFrame({'group': trial_error[:, 0],
                               'userId_grouping': trial_error[:, 1],
                               'explanation': trial_error[:, 2]})
fig = px.bar(trial_error_df,
             x=trial_error_df["group"],
             y=trial_error_df["userId_grouping"],
             color=trial_error_df["group"],
             title='Grouped Rating Distribution')

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()
trial_error_df

**Let us explain what is happening here:**

The user Id's are grouped by the rating counts in a grouping range illustrated in the DataFrame above.

In the Grouped Rating Distribution bar graph, it is visually displayed that there is unequal distribution. The distribution is skewed to the left, with the majority of the user ids in the rating count range between 1 and 50. At the same time, the last group has only a value count of 61, which is a significant difference from group one with a value count of 110 010.

In [ ]:
def user_ratings_count(df, n):
    plt.figure(figsize=(14,7))
    data = df['userId'].value_counts().head(n)
    ax = sns.barplot(x = data.index, y = data, order= data.index, palette='CMRmap', edgecolor="black")
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), fontsize=11, ha='center', va='bottom')
    plt.title(f'Top {n} Users by Number of Ratings', fontsize=14)
    plt.xlabel('User ID')
    plt.ylabel('Number of Ratings')
    plt.show()

user_ratings_count(train,10)

We can see that UserID **72315** shows some form of uneven distribution which is as a result of having too many ratings as compared to others, hence deemed to be an outlier

#### Now we will begin to explore the *Movie* Data for insights

In [ ]:
#Let us merge the train data withh the movie data
movies = pd.merge(train, df_movies,on='movieId',how='inner')
movies.head()

In [ ]:
# We will add an additional data i.e df_imdb data
complete_movie = pd.merge(movies,df_imdb,on='movieId',how='inner')
complete_movie.head()

##### We will convert our timestamp and carry out some visualization, but for now let us continue

In [ ]:
#Get the top ratings of movies
def top_rating_plot(df,column, n):
    plt.figure(figsize=(14,7))
    data = df[str(column)].value_counts().head(n)
    ax = sns.barplot(x = data.index, y = data, order= data.index, palette='CMRmap', edgecolor="black")
    for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), fontsize=11, ha='center', va='bottom')
    plt.title(f'Top {n} {column.title()} by Number of Ratings', fontsize=14)
    plt.xlabel(column.title())
    plt.ylabel('Number of Ratings')
    plt.xticks(rotation=90)
    plt.show()

top_rating_plot(movies,'title',15)

From the observation in the plot, what does this tell us:

  * All the movies in the top 15 by Number of Ratings were released in the 90's with only one  i.e "Lord of the Rings" released in the year 2001

In [ ]:
gc.collect()

### Exploring Movie Genre

In [ ]:
#Get the categories of Movie Genre
movie_genres = pd.DataFrame(df_movies['genres'].str.split("|").tolist(),
                      index=df_movies['movieId']).stack()
movie_genres = movie_genres.reset_index([0, 'movieId'])
movie_genres.columns = ['movieId', 'Genre']
movie_genres.head()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
sns.countplot(x='Genre',
              data=movie_genres,
              palette='CMRmap',
              order=movie_genres['Genre'].
              value_counts().index)
plt.xticks(rotation=90)
plt.xlabel('Genre', size=20)
plt.ylabel('Count', size=20)
plt.title('Distribution of Movie Genres', size=25)
plt.show()

Mere looking at the graph, we can tell that **Drama, Comedy,Thriller and Romance** stand out as the popular movie genres 

Several factors can be attributed to why we these genres stand out, viz:
  * 
  *
  *
  

#### Let us get an interesting wordcloud to showcase movie titles and the count of ratings

In [ ]:
# Wordcloud of movie titles
movie = df_movies['title'] = df_movies['title'].astype('str')
wordcloud = ' '.join(movie)
title_wordcloud = WordCloud(stopwords = STOPWORDS,
                            background_color = 'Black',
                            height = 1200,
                            width = 900).generate(wordcloud)
plt.figure(figsize = (14,7), facecolor=None)
plt.imshow(title_wordcloud)
plt.axis('off')
plt.title('Distribution of words from movie titles')
plt.tight_layout(pad=0)
plt.show()

top_rating_plot(movies,'rating',10)

   * The most common rating we have is 4, followed closely by 3 and 5.

   * The least common seen here is 0.5

#### Let us consider the Timestamps to get some insights 

##### Extracting datetime

In [ ]:
import datetime

In [ ]:
#From movie data, convert timestamp to datetime
movies['time_dt'] = movies['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [ ]:
movies.head(3)

#### Let's also show the number and average value of ratings variation in time.

In [ ]:
movies['year'] = movies['time_dt'].dt.year
movies['month'] = movies['time_dt'].dt.month
movies['day'] = movies['time_dt'].dt.day
movies['dayofweek'] = movies['time_dt'].dt.dayofweek

In [ ]:
movies.head(3)

In [ ]:
dt = movies.groupby(['year'])['rating'].count().reset_index()
fig, (ax) = plt.subplots(ncols=1, figsize=(12,6))
plt.plot(dt['year'],dt['rating']); plt.xlabel('Year'); plt.ylabel('Number of votes'); plt.title('Number of votes per year')
plt.show()

From the line plot, we can see that the number of votes (ratings) peaked between **2015 - 2018**

Can be attributed to several factors like;
   * Quality of movies at the time
   * Genre of movie
   * //////
   etc

In [ ]:
dt = movies.groupby(['year'])['rating'].mean().reset_index()
fig, (ax) = plt.subplots(ncols=1, figsize=(12,6))
plt.plot(dt['year'],dt['rating']); plt.xlabel('Year'); plt.ylabel('Average ratings'); plt.title('Average ratings per year')
plt.show()

## Preprocessing

## Modeling

We would be considering 2 types here:
  * Content-Based Filtering Recommendation
  * Collaborative filtering 

### Content-Based Filtering Recommendation

The Content-Based Recommendation system computes similarity between movies based on movie genres using the selected movie as a baseline.

In [ ]:
## Cosine-similarity

from sklearn.metrics.pairwise import linear_kernel
# Break up the big genre string into a string array
genre = df_movies['genres'].str.split('|')
genre = df_movies['genres'].fillna("").astype('str')
sample_genre = genre.head(20000)   # remove sample on EC2 to acess the whole dataset

#create a Tf_idf vectorizer
tf = TfidfVectorizer(analyzer = 'word',
                     ngram_range = (1, 3),
                     min_df = 0,
                     stop_words = 'english')

vecto = tf.fit_transform(sample_genre)     # replace sample_genre with genre
# cosine similarities
cosine_sim = linear_kernel(vecto, vecto)


The TF-IDF vectorizer was used to find the relationship in terms of relative importance of the movie. The systems make use of Cosine Similarity to compute numeric quantities that highlights the similarities between the movies (one as a baseline).

In [ ]:

# The movie_recommendation function
def movie_recommendations(movie):
    # set the movie title as the new index
    movie_index=pd.Series(df_movies.index,
                          index = df_movies['title'])
    # generate  similarities between the movie title and movie index based on genre
    similarities = list(enumerate(cosine_sim[movie_index[movie]]))
    similarities = sorted(similarities, key = lambda x: x[1],
                          reverse = True)[1:11]
    movie_recommendation = df_movies['title'].iloc[[i[0] for i in similarities]]
    return movie_recommendation

In [ ]:
movie_recommendations('Into the West (2005)')

A function was assembled to estimate top 10 movies that are similar to the base movie using genre as a basis. The function was passed an input (‘Into the West (2005)’), a baseline and generated top 10 similar movies using cosine similarity score computed above.

### Collaborative-Based Filtering Reccomendation
Collaborative filtering addresses some of the limitations of content-based filtering; collaborative filtering uses similarities between users and items simultaneously to provide recommendations.
Collaborative filtering models can recommend an item to **user A** based on the interests of a similar **user B**. 

Furthermore, the embeddings can be learned automatically, without relying on hand-engineering of features.

Taking the first 1  million dataset to train on 

### Splitting Data into Train and Test Set

In [ ]:
#Train and Test set
train.drop('timestamp', axis=1)
train_subset = train[:1000000]
reader = Reader(rating_scale=(train_subset['rating'].min(), train_subset['rating'].max()))
data = Dataset.load_from_df(train_subset[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=42)


In [ ]:
gc.collect()

### Now we will consider several Algorithms to test with

#### 1. SlopeOne Algorithm

Slope One is a family of algorithms used for collaborative filtering. 
It is the simplest form of non-trivial item-based collaborative filtering based on ratings

Their simplicity makes it especially easy to implement them efficiently while their accuracy is often on par with more complicated and computationally expensive algorithms

In [ ]:
slo_model = SlopeOne()
slo_model.fit(trainset)
slo_predictions = slo_model.test(testset)
slo_rmse=accuracy.rmse(slo_predictions)

In [ ]:
gc.collect()

### Non-Negative Matrix Factorization Algorithm
Non-negative matrix factorization, also called non-negative matrix approximation is a group of algorithms in multivariate analysis and linear algebra where a matrix V is factorized into two matrices W and H, with the property that all three matrices have no negative elements.

In [ ]:
nmf_model = NMF(n_epochs=10, n_factors=200, random_state=42,verbose=True)
nmf_model.fit(trainset)
nmf_predictions =nmf_model.test(testset)
nmf_rmse = accuracy.rmse(nmf_predictions)

In [ ]:
gc.collect()

### CoClustering Algorithm
Biclustering, block clustering , co-clustering, or two-mode clustering is a data mining technique which allows simultaneous clustering of the rows and columns of a matrix. 

In [ ]:
cc_model = CoClustering(n_epochs=10,random_state=42)
cc_model.fit(trainset)
cc_predictions = cc_model.test(testset)
cc_rmse=accuracy.rmse(cc_predictions)

In [ ]:
gc.collect()

### Singular Value Decomposition (SVD)
The singular value decomposition (SVD) provides another way to factorize a matrix, into singular vectors and singular values. The SVD allows us to discover some of the same kind of information as the eigendecomposition.The SVD is used widely both in the calculation of other matrix operations, such as matrix inverse, but also as a data reduction method in machine learning. SVD can also be used in least squares linear regression, image compression, and denoising data.

In [ ]:
svd_model = SVD(n_epochs=60,n_factors=400,init_std_dev=0.005,random_state=42,verbose=True)
svd_model.fit(trainset)
svd_predictions = svd_model.test(testset)
svd_rmse = accuracy.rmse(svd_predictions)

In [ ]:
gc.collect()

### Singular Value Decomposition plus-plus (SVDpp)
The SVD++ algorithm, an extension of SVD taking into account implicit ratings.

In [ ]:
svdpp_model = SVDpp(n_epochs=10,n_factors=400,init_std_dev=0.001,random_state=42, verbose=True)
svdpp_model.fit(trainset)
svdpp_predictions = svdpp_model.test(testset)
svdpp_rmse = accuracy.rmse(svdpp_predictions)

In [ ]:
gc.collect()

### BaselineOnly algorithm¶
Algorithm predicting the baseline estimate for given user and item.

In [ ]:
bsl_options = {'method': 'sgd','n_epochs': 10}
blo_model = BaselineOnly(bsl_options=bsl_options,verbose=True)
blo_model.fit(trainset)
blo_predictions = blo_model.test(testset)
# Calculate RMSE
blo_rmse = accuracy.rmse(blo_predictions)

In [ ]:
gc.collect()

## Performance/Evaluation

We built and tested six different collaborative filtering models and compared their performance using a statistical measure known as the root mean squared error (RMSE), which determines the average squared difference between the estimated values and the actual value. A low RMSE value indicates high model accuracy.

#### Let us see which of the algorithms give us the lowest (better RMSE) score

In [ ]:
rmse_scores =[nmf_rmse,slo_rmse,cc_rmse,svd_rmse,svdpp_rmse,blo_rmse]
models =['NMF','SlopeOne','CoClustering','SVD','SVD++','BaselineOnly']

accuracy_data = pd.DataFrame({'model':models,'RMSE':rmse_scores})

In [ ]:
gc.collect()

In [ ]:
accuracy_data.sort_values(by='RMSE')

### Better Visualized in a graph

In [ ]:
fig, ax = plt.subplots(figsize=(14,7))
sns.barplot(data=accuracy_data.sort_values(by='RMSE'), x='model', y='RMSE', palette="CMRmap", edgecolor="black", ax=ax)
ax.set_xlabel("Model")
ax.set_ylabel('RMSE Score')
ax.set_yticklabels(['{:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2, p.get_y() + p.get_height(), round(p.get_height(),3), fontsize=12, ha="center", va='bottom')
plt.title('Model Accuracy By RMSE Score', fontsize=14)
plt.show()

## Saving and Exporting model rating as a csv File

In [ ]:
#read the test data
test_df.head()

In [ ]:
#writing  a function that does the whole process 
predictions = []
for i, row in test_df.iterrows():
    x = (algo.predict(row.userId, row.movieId))
    pred = x[3]
    predictions.append(pred)
test_df['newID'] = test_df['userId'].map(str) +'_'+ test_df['movieId'].map(str)
results = pd.DataFrame({"Id":test_df['newID'],"rating": predictions})
# Saves as CSV 
results.to_csv("Team_4.csv", index=False)

In [ ]:
#View Result
results